In [1]:
!pwd


/Users/shubh/Documents/what-you-threw/research


In [2]:
%cd ..

/Users/shubh/Documents/what-you-threw


In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [4]:
from garb_classifier.constants import *
from garb_classifier.utils.common import read_yaml,create_directories



In [5]:
CONFIG_FILE_PATH

PosixPath('config/config.yaml')

In [6]:
class ConfigurationManager:
    def __init__(self, config_file = CONFIG_FILE_PATH, params_file = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file)
        self.params = read_yaml(params_file)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self):
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config

In [7]:


import os
import urllib.request as request
import zipfile
from garb_classifier import logger
from garb_classifier.utils.common import get_size



In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    #data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-09-19 09:09:54,860: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-09-19 09:09:54,862: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-19 09:09:54,863: INFO: common: created directory at: artifacts]
[2023-09-19 09:09:54,865: INFO: common: created directory at: artifacts/data_ingestion]


In [ ]:
read_yaml('/Users/shubh/Documents/what-you-threw/config/config.yaml')